In [ ]:
from google.colab import files
uploaded = files.upload()

Saving text.txt to text.txt


In [ ]:


def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [ ]:

# load document
in_filename = 'text.txt'
doc = load_doc(in_filename)
print(doc[:200])


quarterly profits at us media giant timewarner jumped % to $amount £123m for the three months to december , from $amount year earlier . the firm , which is now one of the biggest investors in google ,


In [ ]:
import string
 

def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [ ]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 1763


In [ ]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [ ]:
# save sequences to file
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

In [ ]:

from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
 
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
 
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(512, return_sequences=False))
model.add(Dense(vocab_size, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)
 
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Epoch 1/100
1763/1763 [==============================] - 2s 1ms/step - loss: 6.3816 - acc: 0.0459
Epoch 2/100
1763/1763 [==============================] - 1s 584us/step - loss: 6.0241 - acc: 0.0431
Epoch 3/100
1763/1763 [==============================] - 1s 585us/step - loss: 5.9142 - acc: 0.0562
Epoch 4/100
1763/1763 [==============================] - 1s 620us/step - loss: 5.8808 - acc: 0.0562
Epoch 5/100
1763/1763 [==============================] - 1s 573us/step - loss: 5.8708 - acc: 0.0562
Epoch 6/100
1763/1763 [==============================] - 1s 596us/step - loss: 5.8592 - acc: 0.0562
Epoch 7/100
1763/1763 [==============================] - 1s 577us/step - loss: 5.8516 - acc: 0.0562
Epoch 8/100
1763/1763 [==============================] - 1s 583us/step - loss: 5.8351 - acc: 0.0562
Epoch 9/100
1763/1763 [==============================] - 1s 599us/step - loss: 5.8689 - acc: 0.0556
Epoch 10/100
1763/1763 [==============================] - 1s 605us/step - loss: 5.8835 - acc: 0.0516
E

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)

		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)
 

in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

model = load_model('model.h5')
 
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
seed_text = lines[randint(0,len(lines))]

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

has blamed high fuel prices for a drop in profits reporting its results for the three months to december the airline made a pre tax profit of amount compared with a year earlier rod eddington bas chief executive said the results were respectable in a third quarter when fuel costs
